<a href="https://colab.research.google.com/github/Doubles2/All_of_DoubleS2/blob/master/ChuLuo_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

본 저자의 코드에서 hpg는 우리 쪽과 다르기 때문에 제외된 데이터셋을 이용하여 모델 설계를 진행함

참고 : https://github.com/kasuo46/Restaurant_Visitor_Forecasting

import library

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

Data Load

In [2]:
#구글 드라이브에서 업로드
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
X_Train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/dataset/train_data.csv')
Y_Train = pd.read_csv('/gdrive/My Drive/Colab Notebooks//dataset/train_label.csv')
X_Test = pd.read_csv('/gdrive/My Drive/Colab Notebooks//dataset/test_data.csv')
Y_Test = pd.read_csv('/gdrive/My Drive/Colab Notebooks//dataset/test_label.csv')

In [4]:
print('X_Train : ', X_Train.shape)
print('Y_Train : ', Y_Train.shape)
print('X_Test : ', X_Test.shape)
print('Y_Test : ', Y_Test.shape)

X_Train :  (201686, 22)
Y_Train :  (201686, 1)
X_Test :  (50422, 22)
Y_Test :  (50422, 1)


필요 함수 정의

In [0]:
def rmsle(y, pred):
  from sklearn.metrics import mean_squared_log_error
  return mean_squared_log_error(y, pred)**0.5

GBR


In [6]:
model_gbr = GradientBoostingRegressor(learning_rate=0.2, verbose=True)
model_gbr.fit(X_Train, Y_Train)
pred_train_gbr = np.clip(model_gbr.predict(X_Train), a_min=0.0, a_max=None)
pred_val_gbr = np.clip(model_gbr.predict(X_Test), a_min=0.0, a_max=None)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1         228.4059           30.84s
         2         192.9713           29.68s
         3         169.7007           29.25s
         4         154.2997           29.12s
         5         143.8966           28.42s
         6         137.0297           27.83s
         7         132.3163           27.61s
         8         129.0338           27.17s
         9         126.7013           26.97s
        10         125.0511           26.60s
        20         119.6136           22.29s
        30         117.1160           18.75s
        40         116.1330           16.00s
        50         115.2428           13.17s
        60         114.1784           10.45s
        70         113.1615            7.79s
        80         111.4666            5.19s
        90         110.4180            2.57s
       100         110.0481            0.00s


KNR

In [0]:
model_knr = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
model_knr.fit(X_Train, Y_Train)
pred_train_knr = np.clip(model_knr.predict(X_Train), a_min=0.0, a_max=None)
pred_val_knr = np.clip(model_knr.predict(X_Test), a_min=0.0, a_max=None)

average of multiple models

In [0]:
del model_gbr, model_knr, X_Train, X_Test
pred_train_mix = (pred_train_gbr + pred_train_knr)/2
pred_val_mix = (pred_val_gbr + pred_val_knr)/2

fit and predict

In [0]:
print('GradientBoostingRegressor RMSLE: Train/Validation', rmsle(Y_Train, pred_train_gbr), rmsle(Y_Test, pred_val_gbr))
print('KNeighborsRegressor RMSLE: Train/Validation', rmsle(Y_Train, pred_train_knr), rmsle(Y_Test, pred_val_knr))
print('MixedRegressor RMSLE: Train/Validation', rmsle(Y_Train, pred_train_mix), rmsle(Y_Test, pred_val_mix))